## Exercises

In these exercises we will work with genome sequence records for some isolates of the SARS-CoV-2 virus. We will inspect the records, get familiar with their annotations and features. We will then extract nucleic acid sequence for the spike gene and predict the protein sequence.


### 7.1

You are given a text file named "SARS-CoV-2_acc.txt". In this file, Genbank accessions for a number of SARS-CoV-2 isolates from different locations around the world are listed. The accessions are separated by new lines. Open, read and parse this file into a list called `accessions`.

In [44]:
with open("SARS-CoV-2_acc.txt") as infile:
    accessions = infile.read().splitlines()

How many accessions are there? What is the first accession? What is the last accession?

In [45]:
print("There are {} accessions".format(len(accessions)))
print("First accession is:", accessions[0])
print("Last accession is:", accessions[-1])

There are 200 accessions
First accession is: MT385458.1
Last accession is: MT263453.1


### 7.2


**1.** For this and the following exercises we will work only with the **first accession**. Retrieve the sequence record for the first accesssion from Genbank using `Bio.Entrez`. Keep this record as a new variable.

In [46]:
from Bio import Entrez
from Bio import SeqIO

Entrez.email = "bulak.arpat@unil.ch"
handle = Entrez.efetch(db="nucleotide", id=accessions[0], rettype="gb", retmode="text")
rec = SeqIO.read(handle, "genbank")
handle.close()

What is the `id`, `name` and `description` of this record?

In [47]:
print("ID={}\nName={}\nDescription={}".format(
    rec.id, rec.name, rec.description)
)

ID=MT385458.1
Name=MT385458
Description=Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/CA-CZB0706/2020, complete genome


Confirm that the ID of this record matches the first accession in the accessions text file. Based on these information, can you guess in which country this isolate was identified?

In [48]:
print("Accession IDs do match:", rec.id == accessions[0])
print("Country of origin could be", rec.description.split("/")[2])

Accession IDs do match: True
Country of origin could be USA


**2.** How many entries are there in the `annotations` of this record?

In [49]:
print("There are", len(rec.annotations), "annotations associated with this record")

There are 12 annotations associated with this record


Print the 'taxonomy' and the 'references' of this record. What is the title of the publication this sequence was published?

In [50]:
print(rec.annotations["taxonomy"])
print(rec.annotations["references"])
print()
print("Method of submission:", rec.annotations["references"][0].title)

['Viruses', 'Riboviria', 'Orthornavirae', 'Pisuviricota', 'Pisoniviricetes', 'Nidovirales', 'Cornidovirineae', 'Coronaviridae', 'Orthocoronavirinae', 'Betacoronavirus', 'Sarbecovirus']
[Reference(title='Direct Submission', ...)]

Method of submission: Direct Submission


**3.** How many entries are there in the `features` of this record? Print the first feature.

In [51]:
print("There are", len(rec.features), "features")
print(rec.features[0])

There are 55 features
type: source
location: [0:29861](+)
qualifiers:
    Key: collection_date, Value: ['2020-03-23']
    Key: country, Value: ['USA: CA']
    Key: db_xref, Value: ['taxon:2697049']
    Key: host, Value: ['Homo sapiens']
    Key: isolate, Value: ['SARS-CoV-2/human/USA/CA-CZB0706/2020']
    Key: isolation_source, Value: ['nasopharyngeal/oropharyngeal swab']
    Key: mol_type, Value: ['genomic RNA']
    Key: organism, Value: ['Severe acute respiratory syndrome coronavirus 2']



This is a 'source' feature and usually there is a single 'source' feature for a record. It holds like the 'annotations' very useful information about the source of this record. Can you confirm the country of origin?

In [52]:
print("The place of origin for this isolate is", rec.features[0].qualifiers["country"][0])

The place of origin for this isolate is USA: CA


How many different possible values are there for the `type` of these features and what are they?

In [53]:
# Using a list
feature_types = []
for feature in rec.features:
    if feature.type not in feature_types:
        feature_types.append(feature.type)
print("There are {} different values for feature types".format(len(feature_types)))
print(feature_types)

There are 5 different values for feature types
['source', 'gene', 'CDS', 'mat_peptide', 'stem_loop']


In [54]:
# Optional - alternative using set
feature_types_set = set([feature.type for feature in rec.features])
print("There are {} different values for feature types".format(len(feature_types_set)))
print(feature_types_set)

# Can you spot the differences between the list and set versions?
# Sets are very useful counters to hold unique members
# Try help(set)

There are 5 different values for feature types
{'source', 'stem_loop', 'gene', 'mat_peptide', 'CDS'}


How many 'gene's does this viral genome have, according to the features? How many 'CDS's are defined in the features? Are the number of genes and CDSs same?

*Hint*, dictionaries can be used to count multiple things within a single loop.

In [55]:
counter = {'CDS': 0, 'gene': 0}
for feature in rec.features:
    if feature.type in counter:
        counter[feature.type] += 1
print(counter)

{'CDS': 12, 'gene': 11}


### 7.3

In this exercise, we will find the 'gene' and 'CDS' features for the spike protein. The spike protein (S protein) is a large type I transmembrane protein, which is highly glycosylated. Spike proteins assemble into trimers on the virion surface to form the distinctive "corona", or crown-like appearance. The gene is usually called the **S** gene and the protein product is called **surface glycoprotein**.

**1.** First, write a function, which accepts a single argument, a record, which has to be of `SeqRecord` type. This function should loop over all features of this record and return a `list` of features whose `.type` is either **'gene' or 'CDS'**. You will use this function in all other questions of **7.3**!

In [56]:
def get_gene_cds_features(rec):
    result = []
    for feature in rec.features:
        if feature.type in ['CDS', 'gene']:
            result.append(feature)
    return result

Using this function, print the **keys** of the qualifiers for all 'gene' and 'CDS' features. What is the single **key** that can be found in all qualifiers?

*Hint:* Qualifiers are a special kind of dictionary called OrderedDict. Their keys can be accessed by the `.keys()` method.

In [57]:
for feature in get_gene_cds_features(rec):
    print("{}: {}".format(feature.type, list(feature.qualifiers.keys())))

gene: ['gene']
CDS: ['gene', 'ribosomal_slippage', 'codon_start', 'product', 'protein_id', 'translation']
CDS: ['gene', 'codon_start', 'product', 'protein_id', 'translation']
gene: ['gene']
CDS: ['gene', 'codon_start', 'product', 'protein_id', 'translation']
gene: ['gene']
CDS: ['gene', 'codon_start', 'product', 'protein_id', 'translation']
gene: ['gene']
CDS: ['gene', 'codon_start', 'product', 'protein_id', 'translation']
gene: ['gene']
CDS: ['gene', 'codon_start', 'product', 'protein_id', 'translation']
gene: ['gene']
CDS: ['gene', 'codon_start', 'product', 'protein_id', 'translation']
gene: ['gene']
CDS: ['gene', 'codon_start', 'product', 'protein_id', 'translation']
gene: ['gene']
CDS: ['gene', 'codon_start', 'product', 'protein_id', 'translation']
gene: ['gene']
CDS: ['gene', 'codon_start', 'product', 'protein_id', 'translation']
gene: ['gene']
CDS: ['gene', 'codon_start', 'product', 'protein_id', 'translation']
gene: ['gene']
CDS: ['gene', 'codon_start', 'product', 'protein_id', 

**2.**  Now, using the same function, print the `'gene'` qualifier for all. Can you spot the 'S' gene?

In [58]:
for feature in get_gene_cds_features(rec):
    print("{}: {}".format(feature.type, feature.qualifiers['gene']))

gene: ['ORF1ab']
CDS: ['ORF1ab']
CDS: ['ORF1ab']
gene: ['S']
CDS: ['S']
gene: ['ORF3a']
CDS: ['ORF3a']
gene: ['E']
CDS: ['E']
gene: ['M']
CDS: ['M']
gene: ['ORF6']
CDS: ['ORF6']
gene: ['ORF7a']
CDS: ['ORF7a']
gene: ['ORF7b']
CDS: ['ORF7b']
gene: ['ORF8']
CDS: ['ORF8']
gene: ['N']
CDS: ['N']
gene: ['ORF10']
CDS: ['ORF10']


**3.** Using the `'gene'` qualifier from the previous exercise, print the `location` of all 'gene' and 'CDS' features for the 'S' gene. 

*Attention, the value of the `'gene'` qualifier is a `list`*

In [59]:
for feature in get_gene_cds_features(rec):
    if feature.qualifiers['gene'][0] == "S":
        print("{}: {}".format(feature.type, feature.location))

gene: [21549:25371](+)
CDS: [21549:25371](+)


### 7.4

**1.** Create a new record by **slicing** the previous record using the coordinates of the 'S' gene. How many features does this 'S' gene record have?

In [60]:
s_gene = rec[21549:25371]
print("'S' gene record has", len(s_gene.features), "features")

'S' gene record has 2 features


**2.** Translate the 'S' gene (transcript in this case, as this is an RNA virus) into a new variable called `s_protein`. Which translation table should we use (a little biology)? Does the protein sequence contain a stop? If so, try to translate without a stop character. How many amino acids does this protein have?

In [61]:
s_protein = s_gene.translate(to_stop=True)
print("S protein has", len(s_protein), "amino acids")

S protein has 1273 amino acids


### 7.5

**1.** In **"SARS-CoV-2_S.gb"** file, you will find the GenBank sequence records of the 'S' gene for the first 50 accessions we used in previous exercises. Can you create a **fasta** file that contains the spike protein sequences for these? Try to keep their `.id` and avoid translating stop codons!

In [62]:
with open("spike_proteins.fa", "w") as fasta:
    for rec in SeqIO.parse("SARS-CoV-2_S.gb", "genbank"):
        prot = rec.translate(to_stop=True)
        prot.id = rec.id
        SeqIO.write(prot, fasta, "fasta")


**2**. Did you notice the *Warning* above, when we try to translate the first 50 accessions. It seems the length of one or more of our 'S' CDSs is not multiple of three. Can you find which one?

In [63]:
for rec in SeqIO.parse("SARS-CoV-2_S.gb", "genbank"):
    for feature in rec.features:
        if feature.type == 'gene' and 'S' in feature.qualifiers['gene']:
            s_length = feature.location.end - feature.location.start
            if s_length % 3 != 0:
                print("'S' gene from '{}' has a length of {}, which is not a multiple of 3".format(
                    rec.id, s_length))

'S' gene from 'MT444558.1' has a length of 3719, which is not a multiple of 3
